# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_file_location = '../WeatherPy/city_df.csv'
city_df = pd.read_csv(city_file_location)
city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,0,Karratha,95.0,AU,2020-04-20 03:33:50,51.0,-20.74,116.85,87.69,9.84
1,1,Karratha,95.0,AU,2020-04-20 03:33:50,51.0,-20.74,116.85,87.69,9.84
2,2,Dartford,0.0,GB,2020-04-20 03:30:02,81.0,51.44,0.22,46.00,3.36
3,3,Pamplona,90.0,ES,2020-04-20 03:34:34,93.0,42.82,-1.64,53.01,5.82
4,4,Katsuura,75.0,JP,2020-04-20 03:34:34,93.0,35.13,140.30,55.40,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Access maps with unique API key
gmaps.configure(g_key)

In [7]:
# Set parameters for gmaps
locations = city_df[["Lat", "Lon"]]
humidity = city_df["Humidity"].astype(float)

# Plot the heatmap
figure = gmaps.figure(map_type='HYBRID')

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity,
                                 dissipating=False, 
                                 max_intensity=50, 
                                 point_radius=1.5)

# Add the layer to the figure
figure.add_layer(heat_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
conditions_df = city_df.copy()
conditions_df = conditions_df[conditions_df['Max Temp'] >=70]
conditions_df = conditions_df[conditions_df['Max Temp'] <=80]
conditions_df = conditions_df[conditions_df['Wind Speed'] <=5]
conditions_df = conditions_df[conditions_df['Cloudiness'] ==0]
conditions_df = conditions_df.dropna()
conditions_df = conditions_df.reset_index()

# Clean up columns by using drop
conditions_df_drop = conditions_df.drop(['index', 'Unnamed: 0'], axis=1)
conditions_df_drop

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Port Alfred,0.0,ZA,2020-04-20 03:35:26,63.0,-33.59,26.89,71.01,3.00
1,Port Alfred,0.0,ZA,2020-04-20 03:35:26,63.0,-33.59,26.89,71.01,3.00
2,Port Alfred,0.0,ZA,2020-04-20 03:35:26,63.0,-33.59,26.89,71.01,3.00
3,Puerto Quijarro,0.0,BO,2020-04-20 03:38:03,86.0,-17.78,-57.77,73.02,1.81
4,Sawākin,0.0,SD,2020-04-20 03:38:36,43.0,19.11,37.33,76.71,2.77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# Store into variable named hotel_df
hotel_df = pd.DataFrame(conditions_df_drop, columns=["City", "Country", "Lat", "Lon"])

# Add a "Hotel Name" to column in the DataFrame
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Port Alfred,ZA,-33.59,26.89,
1,Port Alfred,ZA,-33.59,26.89,
2,Port Alfred,ZA,-33.59,26.89,
3,Puerto Quijarro,BO,-17.78,-57.77,
4,Sawākin,SD,19.11,37.33,


In [37]:
hotel_name = []

# Params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# For loop
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lon"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # URL Requests
    name_address = requests.get(base_url, params=params)
    
    # Jsonify
    name_address = name_address.json()
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
#Append hotel name       
hotel_df['Hotel Name'] = hotel_name
hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
1,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
2,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
3,Puerto Quijarro,BO,-17.78,-57.77,NaN
4,Sawākin,SD,19.11,37.33,اولاد خضر


In [38]:
#Drop the data missing NaN
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
1,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
2,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
4,Sawākin,SD,19.11,37.33,اولاد خضر


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [40]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))